## Deliverable 2. Create a Customer Travel Destinations Map.

In [78]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [79]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bad Freienwalde,DE,2021-11-01 01:38:07,52.7873,14.0304,53.76,75,100,12.97,overcast clouds
1,1,New Norfolk,AU,2021-11-01 01:38:07,-42.7826,147.0587,64.65,49,100,4.00,overcast clouds
2,2,Tasiilaq,GL,2021-11-01 01:38:08,65.6145,-37.6368,23.16,58,5,8.10,clear sky
3,3,Busselton,AU,2021-11-01 01:33:54,-33.6500,115.3333,59.04,59,70,20.33,broken clouds
4,4,Salalah,OM,2021-11-01 01:38:08,17.0151,54.0924,71.69,60,0,3.44,clear sky


In [80]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [81]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Saint-Georges,GD,2021-11-01 01:38:10,12.0564,-61.7485,80.28,78,40,5.75,scattered clouds
17,17,Hilo,US,2021-11-01 01:38:13,19.7297,-155.0900,80.33,78,40,9.22,scattered clouds
19,19,Avarua,CK,2021-11-01 01:35:41,-21.2078,-159.7750,77.05,69,75,17.27,broken clouds
25,25,Butaritari,KI,2021-11-01 01:38:16,3.0707,172.7902,82.24,74,2,10.69,light rain
28,28,Bodden Town,KY,2021-11-01 01:38:08,19.2833,-81.2500,82.40,75,75,5.01,broken clouds
38,38,Sulya,IN,2021-11-01 01:38:22,12.5667,75.3833,75.24,92,79,2.35,broken clouds
39,39,Sao Filipe,CV,2021-11-01 01:38:22,14.8961,-24.4956,79.56,76,99,7.63,overcast clouds
43,43,Bathsheba,BB,2021-11-01 01:38:24,13.2167,-59.5167,75.81,88,75,17.27,broken clouds
51,51,Kirakira,SB,2021-11-01 01:39:28,-10.4544,161.9205,86.68,63,93,15.03,overcast clouds
55,55,The Valley,AI,2021-11-01 01:38:24,18.2170,-63.0578,82.09,70,1,13.42,clear sky


In [82]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [83]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [84]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Saint-Georges,GD,80.28,scattered clouds,12.0564,-61.7485,
17,Hilo,US,80.33,scattered clouds,19.7297,-155.0900,
19,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,
25,Butaritari,KI,82.24,light rain,3.0707,172.7902,
28,Bodden Town,KY,82.40,broken clouds,19.2833,-81.2500,
38,Sulya,IN,75.24,broken clouds,12.5667,75.3833,
39,Sao Filipe,CV,79.56,overcast clouds,14.8961,-24.4956,
43,Bathsheba,BB,75.81,broken clouds,13.2167,-59.5167,
51,Kirakira,SB,86.68,overcast clouds,-10.4544,161.9205,
55,The Valley,AI,82.09,clear sky,18.2170,-63.0578,


In [85]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping")
    
hotel_df

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Saint-Georges,GD,80.28,scattered clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
17,Hilo,US,80.33,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
19,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,Paradise Inn
25,Butaritari,KI,82.24,light rain,3.0707,172.7902,Isles Sunset Lodge
28,Bodden Town,KY,82.40,broken clouds,19.2833,-81.2500,Will T Place
...,...,...,...,...,...,...,...
684,Lae,PG,85.44,light rain,-6.7221,146.9847,Phils Hotel
690,Emilio Carranza,MX,76.44,broken clouds,19.9711,-96.6114,casa carranza
694,Cockburn Town,TC,82.06,overcast clouds,21.4612,-71.1419,Osprey Beach Hotel
703,Byron Bay,AU,75.15,overcast clouds,-28.6500,153.6167,Backpackers Inn on the Beach


In [88]:
#load dataframe back into the notebook so it has NaN. this is a workaround, idk why it wasn't working without reloading it back in

output_data_file = "dirtyhotel.csv"
hotel_df.to_csv(output_data_file, index_label="City_ID")

toclean_hotel_df = pd.read_csv("../Vacation_Search/dirtyhotel.csv")

In [94]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = toclean_hotel_df.dropna()
clean_hotel_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,8,Saint-Georges,GD,80.28,scattered clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
1,17,Hilo,US,80.33,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
2,19,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,Paradise Inn
3,25,Butaritari,KI,82.24,light rain,3.0707,172.7902,Isles Sunset Lodge
4,28,Bodden Town,KY,82.40,broken clouds,19.2833,-81.2500,Will T Place


In [90]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [91]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [92]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))